In [20]:
import spotify_config
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from functions import *

In [21]:
SPOTIPY_CLIENT_ID=spotify_config.SPOTIPY_CLIENT_ID
SPOTIPY_CLIENT_SECRET=spotify_config.SPOTIPY_CLIENT_SECRET


#--------------------------------------------
# RapCaviar Playlist (Import Playlist Here)
# url = "https://open.spotify.com/playlist/37i9dQZF1DX0XUsuxWHRQd"

# Pop Playlist 2021
# url = "https://open.spotify.com/playlist/02ES3fS2eHdLnDEuEUD60V"

# Hot Country
url = 'https://open.spotify.com/playlist/37i9dQZF1DX1lVhptIYRda'
#--------------------------------------------

output_name = 'country'

In [14]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

artists = []
results = sp.playlist_items(url)
for idx, item in enumerate(results['items']):
    track = item['track']
    artists.append(track['artists'][0]['name'])
#     print(idx, track['artists'][0]['name'], " - ", track['name'])
result = [] 
[result.append(x) for x in artists if x not in result] 
print(result)

['Miranda Lambert', 'Jon Pardi', 'Scotty McCreery', 'Morgan Wade', 'Old Dominion', 'Gabby Barrett', 'Jimmie Allen', 'MacKenzie Porter', 'Travis Denning', 'Walker Hayes', 'Maren Morris', 'Cole Swindell', 'Russell Dickerson', 'Morgan Wallen', 'Maddie & Tae', 'Dierks Bentley', 'Thomas Rhett', 'Erin Kinsey', 'Corey Kent', 'Halle Kearns', 'Mitchell Tenpenny', 'Dustin Lynch', 'Carly Pearce', 'Bailey Zimmerman', 'Ingrid Andress', 'Cody Johnson', 'Lee Brice', 'Carrie Underwood', 'Dylan Scott', 'Zach Bryan', 'Conner Smith', 'Hailey Whitters', 'Nate Smith', 'Spencer Crandall', 'Jessie James Decker', 'Chayce Beckham', 'Jason Aldean', 'BRELAND', 'Kelsea Ballerini', 'Luke Combs', 'Kameron Marlowe', 'Restless Road', 'Carter Faith', 'Chase Rice', 'Midland', 'Caitlyn Smith', 'Kane Brown', 'David J', 'ERNEST', 'Danielle Bradbery', 'Dalton Dover', 'Seaforth']


In [ ]:
all_songs = []
for artist_name in result:
    songs = request_song_url(artist_name, 10)
    lyrics = [""] * 10
    index = 0
    for song in songs:
        soup = BeautifulSoup(requests.get(song).content, 'lxml')
        lyrics[index] = ""

        for tag in soup.select('div[class^="Lyrics__Container"], .song_body-lyrics p'):

            for i in tag.select('i'):
                i.unwrap()
            tag.smooth()

            t = tag.get_text(strip=True, separator='\n')

            if t:
                lyrics[index]+=(t)
        print(index)
        index = index + 1
    search_term = artist_name
    genius_search_url = f"http://api.genius.com/search?q={search_term}&access_token={client_access_token}"
    response = requests.get(genius_search_url)
    json_data = response.json()
    for i in range(len(lyrics)):
        lyrics[i]= clean_lyrics(lyrics[i])
    print(lyrics[0])
    index = 0
    for song in json_data['response']['hits']:
        all_songs.append([search_term, song['result']['full_title'], lyrics[index]])
        index += 1

In [23]:
song = pd.DataFrame(all_songs)
song.columns = ['artist_name', 'song_title', 'lyrics']
song.to_csv('datasets/' + output_name + '.csv', index=False)